In [120]:
import matplotlib.pyplot as plt

from PIL import Image
import torch
from torch import nn
from torch import optim
from torchvision import datasets,transforms,models

In [115]:
data_dir = 'all_data'
# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=1)


In [116]:
model = models.densenet121(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [117]:
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                            ('fc1',nn.Linear(1024,512)),
                            ('relu1',nn.ReLU()),
                            ('fc2',nn.Linear(512,128)),
                            ('relu2',nn.ReLU()),
                            ('fc3',nn.Linear(128,2)),
                            ('output',nn.LogSoftmax(dim=1))
                           ]))
model.classifier = classifier
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [119]:
epochs = 5
steps = 0
print_every = 5
running_loss = 0
for i in range(epochs):
    for image, labels in trainloader:
        steps+=1
        
        # load the images and labels to GPU
        image = image.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        output = model(image)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps%print_every==0:         
            acc = 0
            test_loss = 0
            model.eval()
            with torch.no_grad():
                for image, labels in testloader:
                    image = image.to(device)
                    labels = labels.to(device)
                    output = model(image)
                    
                    batch_loss = criterion(output, labels)                    
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(output)
                    
                    top_p, top_label = ps.topk(1,dim=1)
                    equals = top_label==labels.view(*top_label.shape)
                    acc += torch.mean(equals.type(torch.FloatTensor)).item()
            print(f"Epoch {i+1}/{i}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {acc/len(testloader):.3f}")
            running_loss =0
            model.train()

Epoch 1/0.. Train loss: 0.619.. Test loss: 0.431.. Test accuracy: 0.972
Epoch 2/1.. Train loss: 0.433.. Test loss: 0.225.. Test accuracy: 1.000
Epoch 3/2.. Train loss: 0.281.. Test loss: 0.091.. Test accuracy: 0.987
Epoch 4/3.. Train loss: 0.203.. Test loss: 0.065.. Test accuracy: 0.972
Epoch 5/4.. Train loss: 0.259.. Test loss: 0.055.. Test accuracy: 0.968


In [111]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw()
path = askopenfilename() # show an "Open" dialog box and return the path to the selected file


In [112]:
loader = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

def image_loader(image_name):
    image = Image.open(image_name)
    image = loader(image).float() 
    return image.cuda()  #assumes that you're using GPU

image = image_loader(path)
image = image.view(1,3,224,224)
with torch.no_grad():
    output = model(image)
    ps = torch.exp(output)
    top_p, top_label = ps.topk(1,dim=1)
print(top_label)                    

tensor([[1]], device='cuda:0')
